In [1]:
global datadir "/export/storage_adgandhi/MiscLi/VaccineDemandLiGandhi/Data"
global codedir /mnt/staff/zhli/VaxDemandDistance

import delim $datadir/Raw/MAR01.csv, clear

gen shares = vaxfull
replace shares = 0.05 if shares < 0.05
replace shares = 0.95 if shares > 0.95
gen shares_out = 1 - shares // outside shares

gen logshares = log(shares)
gen logshares_out = log(shares_out)
gen delta = logshares - logshares_out
save $datadir/MAR01_vars, replace
global controlvars race_black race_asian race_hispanic race_other ///
	health_employer health_medicare health_medicaid health_other collegegrad ///
    unemployment poverty medianhhincome medianhomevalue popdensity population




(encoding automatically selected: ISO-8859-1)
(44 vars, 1,750 obs)


(104 real changes made)

(159 real changes made)





file /export/storage_adgandhi/MiscLi/VaccineDemandLiGandhi/Data/MAR01_vars.dta
    saved



In [2]:
des, f
// 1. original model
reg vaxfull c.logdistnearest##b4.hpiquartile $controlvars, robust 
loc rep "" // "q" or "a" or "" or "q1"
preserve
if "`rep'" == "q" {
// replace controls with means within hpi quartile
	foreach vv of varlist $controlvars{
		qui bysort hpiquartile: egen mean_`vv' = mean(`vv')
		qui replace `vv' = mean_`vv'
	}
}
else if "`rep'" == "a" {
// replace controls with means across all observations
	foreach vv of varlist $controlvars{
		qui egen mean_`vv' = mean(`vv')
		qui replace `vv' = mean_`vv'
	}
}
else if "`rep'" == "q1" {
	keep if hpiquartile == 1
}

margins [aweight=population], at(logdistnearest=(-0.5(0.1)2.5)) over(hpiquartile) saving(mall, replace)
combomarginsplot mall, labels("HPI Quartile 1" "HPI Quartile 2" "HPI Quartile 3" "HPI Quartile 4") ///
	ciopt(recast(rarea) color(%25)) plotopt(msize(vsmall)) ///
	graphregion(margin(b-4 l+1)) title("") ///
	legend(size(small) region(color(none))) ///
	subtitle("Adjusted Vaccination" "Coverage", size(small) position(11) ring(1) span margin(l=-3 b=2.5) justification(center)) ///
	ylabel(.5 "50%" .6 "60%" .7 "70%" .8 "80%" .9 "90%", labsize(small)) ytitle("") ///
	xtitle("Kilometers to Nearest Vaccination Site (log scale)", size(small)) ///
	xlabel(-.5 ".6" 0 "1" .693 "2" 1.609 "5" 2.303 "10", labsize(small))	
graph export "$codedir/plots/marg1rep`rep'.png", replace
restore
// 2. logit
reg delta c.logdistnearest##b4.hpiquartile $controlvars, robust 
// use the logit command for easier margins
use $datadir/MAR01_vars, clear
expand 2, gen(y)
gen wgt = population * shares if y == 1
replace wgt = population * shares_out if y == 0
replace wgt = round(wgt, 1)
logit y c.logdistnearest##b4.hpiquartile $controlvars [fweight=wgt], vce(cluster zip)
qui margins [aweight=population], at(logdistnearest=(-0.5(0.1)2.5)) over(hpiquartile) saving(mall, replace)
qui combomarginsplot mall, labels("HPI Quartile 1" "HPI Quartile 2" "HPI Quartile 3" "HPI Quartile 4") ///
	ciopt(recast(rarea) color(%25)) plotopt(msize(vsmall)) ///
	graphregion(margin(b-4 l+1)) title("") ///
	legend(size(small) region(color(none))) ///
	subtitle("Adjusted Vaccination" "Coverage", size(small) position(11) ring(1) span margin(l=-3 b=2.5) justification(center)) ///
	ylabel(.5 "50%" .6 "60%" .7 "70%" .8 "80%" .9 "90%", labsize(small)) ytitle("") ///
	xtitle("Kilometers to Nearest Vaccination Site (log scale)", size(small)) ///
	xlabel(-.5 ".6" 0 "1" .693 "2" 1.609 "5" 2.303 "10", labsize(small))	
graph export "$codedir/plots/marg2`rep'.png", replace
// TODO: verify equivalence between logit methods (reg delta ... vs logit y ...)
binscatter2 delta logdistnearest, controls(race_black race_asian race_hispanic race_other health_employer health_medicare health_medicaid health_other collegegrad unemployment poverty medianhhincome medianhomevalue popdensity population) savegraph("/mnt/staff/zhli/binsc_delta.png") replace
binscatter2 logshares logdistnearest, controls(race_black race_asian race_hispanic race_other health_employer health_medicare health_medicaid health_other collegegrad unemployment poverty medianhhincome medianhomevalue popdensity population) savegraph("/mnt/staff/zhli/binsc_logshares.png") replace


Contains data from /export/storage_adgandhi/MiscLi/VaccineDemandLiGandhi/Data/MA
> R01_vars.dta
 Observations:         1,750                  
    Variables:            49                  26 May 2023 10:55
--------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
--------------------------------------------------------------------------------
v1              int     %8.0g                 
date            str10   %10s                  Date
zip             long    %12.0g                Zip
vaccineequityquartile
                str2    %9s                   VaccineEquityQuartile
pop12up         float   %9.0g                 Pop12up
popfull         long    %12.0g                PopFull
poppart         long    %12.0g                PopPart
vaxfull         float   %9.0g                 VaxFull
vaxpartfull     float   %9.0g                 VaxPartFull
x               

In [3]:
// 1. original model
reg vaxfull c.logdistnearest##b4.hpiquartile $controlvars, robust 
loc rep "" // "q" or "a" or "" or "q1"
preserve
if "`rep'" == "q" {
// replace controls with means within hpi quartile
	foreach vv of varlist $controlvars{
		qui bysort hpiquartile: egen mean_`vv' = mean(`vv')
		qui replace `vv' = mean_`vv'
	}
}
else if "`rep'" == "a" {
// replace controls with means across all observations
	foreach vv of varlist $controlvars{
		qui egen mean_`vv' = mean(`vv')
		qui replace `vv' = mean_`vv'
	}
}
else if "`rep'" == "q1" {
	keep if hpiquartile == 1
}

margins [aweight=population], at(logdistnearest=(-0.5(0.1)2.5)) over(hpiquartile) saving(mall, replace)
combomarginsplot mall, labels("HPI Quartile 1" "HPI Quartile 2" "HPI Quartile 3" "HPI Quartile 4") ///
	ciopt(recast(rarea) color(%25)) plotopt(msize(vsmall)) ///
	graphregion(margin(b-4 l+1)) title("") ///
	legend(size(small) region(color(none))) ///
	subtitle("Adjusted Vaccination" "Coverage", size(small) position(11) ring(1) span margin(l=-3 b=2.5) justification(center)) ///
	ylabel(.5 "50%" .6 "60%" .7 "70%" .8 "80%" .9 "90%", labsize(small)) ytitle("") ///
	xtitle("Kilometers to Nearest Vaccination Site (log scale)", size(small)) ///
	xlabel(-.5 ".6" 0 "1" .693 "2" 1.609 "5" 2.303 "10", labsize(small))	
graph export "$codedir/plots/marg1rep`rep'.png", replace
restore
// 2. logit
reg delta c.logdistnearest##b4.hpiquartile $controlvars, robust 
// use the logit command for easier margins
use $datadir/MAR01_vars, clear
expand 2, gen(y)
gen wgt = population * shares if y == 1
replace wgt = population * shares_out if y == 0
replace wgt = round(wgt, 1)
logit y c.logdistnearest##b4.hpiquartile $controlvars [fweight=wgt], vce(cluster zip)
qui margins [aweight=population], at(logdistnearest=(-0.5(0.1)2.5)) over(hpiquartile) saving(mall, replace)
qui combomarginsplot mall, labels("HPI Quartile 1" "HPI Quartile 2" "HPI Quartile 3" "HPI Quartile 4") ///
	ciopt(recast(rarea) color(%25)) plotopt(msize(vsmall)) ///
	graphregion(margin(b-4 l+1)) title("") ///
	legend(size(small) region(color(none))) ///
	subtitle("Adjusted Vaccination" "Coverage", size(small) position(11) ring(1) span margin(l=-3 b=2.5) justification(center)) ///
	ylabel(.5 "50%" .6 "60%" .7 "70%" .8 "80%" .9 "90%", labsize(small)) ytitle("") ///
	xtitle("Kilometers to Nearest Vaccination Site (log scale)", size(small)) ///
	xlabel(-.5 ".6" 0 "1" .693 "2" 1.609 "5" 2.303 "10", labsize(small))	
graph export "$codedir/plots/marg2`rep'.png", replace
// TODO: verify equivalence between logit methods (reg delta ... vs logit y ...)
binscatter2 delta logdistnearest, controls(race_black race_asian race_hispanic race_other health_employer health_medicare health_medicaid health_other collegegrad unemployment poverty medianhhincome medianhomevalue popdensity population) savegraph("/mnt/staff/zhli/binsc_delta.png") replace
binscatter2 logshares logdistnearest, controls(race_black race_asian race_hispanic race_other health_employer health_medicare health_medicaid health_other collegegrad unemployment poverty medianhhincome medianhomevalue popdensity population) savegraph("/mnt/staff/zhli/binsc_logshares.png") replace


Linear regression                               Number of obs     =      1,750
                                                F(22, 1727)       =      51.18
                                                Prob > F          =     0.0000
                                                R-squared         =     0.4082
                                                Root MSE          =     .18865

-------------------------------------------------------------------------------
              |               Robust
      vaxfull | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
--------------+----------------------------------------------------------------
logdistnear~t |   -.002606   .0093838    -0.28   0.781    -.0210107    .0157988
              |
  hpiquartile |
           1  |   -.036524   .0313164    -1.17   0.244     -.097946    .0248979
           2  |   .0050437    .023254     0.22   0.828    -.0405653    .0506526
           3  |   .0134216   .0175714     0.76   0.445

In [23]:
loc rep "" // "q" or "a" or "" or "q1"
preserve
if "`rep'" == "q" {
// replace controls with means within hpi quartile
	foreach vv of varlist $controlvars{
		qui bysort hpiquartile: egen mean_`vv' = mean(`vv')
		qui replace `vv' = mean_`vv'
	}
}
else if "`rep'" == "a" {
// replace controls with means across all observations
	foreach vv of varlist $controlvars{
		qui egen mean_`vv' = mean(`vv')
		qui replace `vv' = mean_`vv'
	}
}
else if "`rep'" == "q1" {
	keep if hpiquartile == 1
}

margins [aweight=population], at(logdistnearest=(-0.5(0.1)2.5)) over(hpiquartile) saving(mall, replace)
combomarginsplot mall, labels("HPI Quartile 1" "HPI Quartile 2" "HPI Quartile 3" "HPI Quartile 4") ///
	ciopt(recast(rarea) color(%25)) plotopt(msize(vsmall)) ///
	graphregion(margin(b-4 l+1)) title("") ///
	legend(size(small) region(color(none))) ///
	subtitle("Adjusted Vaccination" "Coverage", size(small) position(11) ring(1) span margin(l=-3 b=2.5) justification(center)) ///
	ylabel(.5 "50%" .6 "60%" .7 "70%" .8 "80%" .9 "90%", labsize(small)) ytitle("") ///
	xtitle("Kilometers to Nearest Vaccination Site (log scale)", size(small)) ///
	xlabel(-.5 ".6" 0 "1" .693 "2" 1.609 "5" 2.303 "10", labsize(small))	
graph export "$codedir/plots/marg1rep`rep'.png", replace
restore
// 2. logit
reg delta c.logdistnearest##b4.hpiquartile $controlvars, robust 
// use the logit command for easier margins
use $datadir/MAR01_vars, clear
expand 2, gen(y)
gen wgt = population * shares if y == 1
replace wgt = population * shares_out if y == 0
replace wgt = round(wgt, 1)
logit y c.logdistnearest##b4.hpiquartile $controlvars [fweight=wgt], vce(cluster zip)
qui margins [aweight=population], at(logdistnearest=(-0.5(0.1)2.5)) over(hpiquartile) saving(mall, replace)
qui combomarginsplot mall, labels("HPI Quartile 1" "HPI Quartile 2" "HPI Quartile 3" "HPI Quartile 4") ///
	ciopt(recast(rarea) color(%25)) plotopt(msize(vsmall)) ///
	graphregion(margin(b-4 l+1)) title("") ///
	legend(size(small) region(color(none))) ///
	subtitle("Adjusted Vaccination" "Coverage", size(small) position(11) ring(1) span margin(l=-3 b=2.5) justification(center)) ///
	ylabel(.5 "50%" .6 "60%" .7 "70%" .8 "80%" .9 "90%", labsize(small)) ytitle("") ///
	xtitle("Kilometers to Nearest Vaccination Site (log scale)", size(small)) ///
	xlabel(-.5 ".6" 0 "1" .693 "2" 1.609 "5" 2.303 "10", labsize(small))	
graph export "$codedir/plots/marg2`rep'.png", replace
// TODO: verify equivalence between logit methods (reg delta ... vs logit y ...)
binscatter2 delta logdistnearest, controls(race_black race_asian race_hispanic race_other health_employer health_medicare health_medicaid health_other collegegrad unemployment poverty medianhhincome medianhomevalue popdensity population) savegraph("/mnt/staff/zhli/binsc_delta.png") replace
binscatter2 logshares logdistnearest, controls(race_black race_asian race_hispanic race_other health_employer health_medicare health_medicaid health_other collegegrad unemployment poverty medianhhincome medianhomevalue popdensity population) savegraph("/mnt/staff/zhli/binsc_logshares.png") replace








Predictive margins                                       Number of obs = 1,750
Model VCE: Robust

Expression: Linear prediction, predict()
Over:       hpiquartile
1._at: 1.hpiquartile
            logdistnearest = -.5
       2.hpiquartile
            logdistnearest = -.5
       3.hpiquartile
            logdistnearest = -.5
       4.hpiquartile
            logdistnearest = -.5
2._at: 1.hpiquartile
            logdistnearest = -.4
       2.hpiquartile
            logdistnearest = -.4
       3.hpiquartile
            logdistnearest = -.4
       4.hpiquartile
            logdistnearest = -.4
3._at: 1.hpiquartile
            logdistnearest = -.3
       2.hpiquartile
            logdistnearest = -.3
       3.hpiquartile
            logdistnearest = -.3
       4.hpiquartile
            logdistnearest = -.3
4._at: 1.hpiquartile
            logdistnearest = -.2
       2.hpiquartile
            logdistnearest = -.2
       3.hpiquartile
            logdistnearest = -.2
       4.hpiquartile

In [114]:
// 2. logit
reg delta c.logdistnearest##b4.hpiquartile $controlvars, robust 
// use the logit command for easier margins
use $datadir/MAR01_vars, clear
expand 2, gen(y)
gen wgt = population * shares if y == 1
replace wgt = population * shares_out if y == 0
replace wgt = round(wgt, 1)
logit y c.logdistnearest##b4.hpiquartile $controlvars [fweight=wgt], vce(cluster zip)
qui margins [aweight=population], at(logdistnearest=(-0.5(0.1)2.5)) over(hpiquartile) saving(mall, replace)
qui combomarginsplot mall, labels("HPI Quartile 1" "HPI Quartile 2" "HPI Quartile 3" "HPI Quartile 4") ///
	ciopt(recast(rarea) color(%25)) plotopt(msize(vsmall)) ///
	graphregion(margin(b-4 l+1)) title("") ///
	legend(size(small) region(color(none))) ///
	subtitle("Adjusted Vaccination" "Coverage", size(small) position(11) ring(1) span margin(l=-3 b=2.5) justification(center)) ///
	ylabel(.5 "50%" .6 "60%" .7 "70%" .8 "80%" .9 "90%", labsize(small)) ytitle("") ///
	xtitle("Kilometers to Nearest Vaccination Site (log scale)", size(small)) ///
	xlabel(-.5 ".6" 0 "1" .693 "2" 1.609 "5" 2.303 "10", labsize(small))	
graph export "$codedir/plots/marg2`rep'.png", replace
// TODO: verify equivalence between logit methods (reg delta ... vs logit y ...)
binscatter2 delta logdistnearest, controls(race_black race_asian race_hispanic race_other health_employer health_medicare health_medicaid health_other collegegrad unemployment poverty medianhhincome medianhomevalue popdensity population) savegraph("/mnt/staff/zhli/binsc_delta.png") replace
binscatter2 logshares logdistnearest, controls(race_black race_asian race_hispanic race_other health_employer health_medicare health_medicaid health_other collegegrad unemployment poverty medianhhincome medianhomevalue popdensity population) savegraph("/mnt/staff/zhli/binsc_logshares.png") replace


Linear regression                               Number of obs     =      3,500
                                                F(22, 3477)       =      93.18
                                                Prob > F          =     0.0000
                                                R-squared         =     0.3776
                                                Root MSE          =     1.0811

-------------------------------------------------------------------------------
              |               Robust
        delta | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
--------------+----------------------------------------------------------------
logdistnear~t |  -.0144082   .0393568    -0.37   0.714     -.091573    .0627566
              |
  hpiquartile |
           1  |  -.1958131    .127042    -1.54   0.123    -.4448976    .0532713
           2  |  -.0271891   .0954419    -0.28   0.776     -.214317    .1599387
           3  |  -.0241762   .0727011    -0.33   0.739